# GrowthMonitor Wafer Heatmap Visualizer

This notebook visualizes the wafer dose heatmaps produced by `GrowthMonitor`.

Expected CSV format (from `GrowthMonitor_<RUN_ID>.csv`):

- `job_index`
- `wafer_index`
- `row`
- `col`
- `t_end_s`
- `dose_arb`

You can:
1. Point to a CSV file.
2. Choose a `job_index`.
3. Plot the 2D dose map as a heatmap.


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (6, 6)
plt.rcParams['image.origin'] = 'lower'  # row 0 at bottom for wafer view


ImportError: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

## 1. Point to your GrowthMonitor CSV

Update `csv_path` below if needed. You can copy the file into the same folder
as this notebook or give an absolute path.


In [ ]:
# Path to your GrowthMonitor CSV file
run_id = "sim_debug"

csv_path = f"./{run_id}/GrowthMonitor_{run_id}.csv"  # change this to your actual file

if not os.path.exists(csv_path):
    print(f"Warning: CSV file not found at {csv_path}. Update the path above.")
else:
    print(f"Using CSV file: {csv_path}")

df = None
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print("Loaded rows:", len(df))
    print("Columns:", list(df.columns))
    print("Unique job_index values:", sorted(df['job_index'].unique()))


## 2. Choose a job and build the 2D dose map

- Set `job_index_to_plot`.
- The code will infer `gridN` from the max row/col.
- Non-wafer cells (mask==0) will be NaN and appear blank.


In [ ]:
if df is None:
    raise RuntimeError("DataFrame df is not loaded. Fix csv_path and re-run the previous cell.")

# Choose which job to visualize
job_index_to_plot = 0  # change this to any valid job_index

job_df = df[df["job_index"] == job_index_to_plot].copy()
if job_df.empty:
    raise ValueError(f"No rows found for job_index={job_index_to_plot}")

# Infer grid size
max_row = int(job_df["row"].max())
max_col = int(job_df["col"].max())
gridN_row = max_row + 1
gridN_col = max_col + 1
if gridN_row != gridN_col:
    print("Warning: grid is not square; using row dimension as gridN.")
gridN = gridN_row

print(f"Inferred grid size: {gridN} x {gridN}")

# Build arrays
dose = np.full((gridN, gridN), np.nan, dtype=float)

for _, row in job_df.iterrows():
    r = int(row["row"])
    c = int(row["col"])
    val = float(row["dose_arb"])
    dose[r, c] = val

print("Dose stats (ignoring NaN):")
print("  min:", np.nanmin(dose))
print("  max:", np.nanmax(dose))
print("  mean:", np.nanmean(dose))


## 3. Plot the wafer heatmap

- Uses a simple colormap.
- Non-wafer cells should appear blank or as the background color.


In [ ]:
if np.all(np.isnan(dose)):
    raise RuntimeError("Dose array is all NaN. Check that the CSV has non-zero dose_arb values.")

fig, ax = plt.subplots()
im = ax.imshow(dose, interpolation='nearest')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label("dose_arb")

ax.set_title(f"GrowthMonitor heatmap — job_index={job_index_to_plot}")
ax.set_xlabel("col")
ax.set_ylabel("row")

plt.tight_layout()
plt.show()


## 4. Optional: compare multiple jobs

You can repeat the selection + plotting cells for different `job_index` values
or wrap them in a loop to generate a figure per job.
